# Running Design of Experiments (DOE) using CSVGenerator

In this example, the `driver_config.yaml` file is set-up to run a design of experiments for different combinations of solar and electrolyzer capacities using the "csvgen" generator method.

```yaml
general:
  folder_output: ex_20_out
  create_om_reports: true
driver:
  design_of_experiments:
    flag: true
    debug_print: true
    generator: "csvgen"
    filename: "csv_doe_cases.csv"
design_variables:
  solar:
    capacity_kWdc:
      flag: true
      lower: 5000.0
      upper: 5000000.0
      units: "kW"
  electrolyzer:
    n_clusters:
      flag: true
      lower: 1
      upper: 25
      units: "unitless"
```

The different combinations of solar and electrolzyer capacities are listed in the csv file "csv_doe_cases.csv":

```text
solar.capacity_kWdc,electrolyzer.n_clusters
25000.0,5.0
50000.0,5.0
100000.0,5.0
200000.0,5.0
50000.0,10.0
100000.0,10.0
200000.0,10.0
250000.0,10.0
300000.0,10.0
500000.0,10.0
```

In [1]:
from h2integrate.core.h2integrate_model import H2IntegrateModel

/Users/egrant/Documents/projects/GreenHEART/examples/20_solar_electrolyzer_doe/log/hybrid_systems_2025-10-24T17.27.59.955461.log


/Users/egrant/opt/anaconda3/envs/h2i_env/lib/python3.11/site-packages/fastkml/__init__.py:28: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound


### Model set-up and attempted run

We initialize the H2Integrate model and attempt to run the model. A UserWarning will be thrown saying that something may be wrong with our .csv file.

In [2]:
# Create a H2Integrate model
model = H2IntegrateModel("20_solar_electrolyzer_doe.yaml")
# Run the model
model.run()

UserWarning: There may be issues with the csv file csv_doe_cases.csv, which may cause errors within OpenMDAO. To check this csv file or create a new one, run the function h2integrate.core.utilities.check_file_format_for_csv_generator().

### Fixing the bug
The UserWarning tells us that there may be an issue with our csv file. We will use the recommended method to create a new csv file that doesn't have formatting issues.

We'll take the following steps to try and fix the bug:
1. Run the `check_file_format_for_csv_generator` method mentioned in the UserWarning and create a new csv file that is hopefully free of errors
2. Make a new driver config file that has "filename" point to the new csv file created in Step 1.
3. Make a new top-level config file that points to the updated driver config file created in Step 2.

In [3]:
# Import necessary methods and packages
from pathlib import Path
from h2integrate.core.utilities import check_file_format_for_csv_generator
from h2integrate.core.inputs.validation import load_driver_yaml, write_yaml
from hopp.utilities.utilities import load_yaml
from h2integrate.core.dict_utils import update_defaults

# load the driver config file
driver_config = load_driver_yaml("driver_config.yaml")
# specify the filepath to the csv file
csv_fpath = Path(driver_config["driver"]["design_of_experiments"]["filename"]).absolute()
# run the csv checker method, we want it to write the csv file to a new filepath so
# set overwrite_file=False
new_csv_filename = check_file_format_for_csv_generator(
    csv_fpath, driver_config, check_only=False, overwrite_file=False
)
# lets see what the filename of the new csv file is:
new_csv_filename.name

'csv_doe_cases1.csv'

In [4]:
# update the csv filename in the driver config dictionary
updated_driver = update_defaults(driver_config["driver"], "filename", new_csv_filename.name)
driver_config["driver"].update(updated_driver)

# save the updated driver to a new file
write_yaml(driver_config, "driver_config_new.yaml")

# update the driver config filename in the top-level config
main_config = load_yaml("20_solar_electrolyzer_doe.yaml")
main_config["driver_config"] = "driver_config_new.yaml"

# save the updated top-level config file to a new file
write_yaml(main_config, "new_20_solar_electrolyzer_doe.yaml")

### Re-running

Now that we've completed the debugging and fixing steps, lets try to run the simulation again but with our new files.

In [5]:
# Create a H2Integrate model
model = H2IntegrateModel("new_20_solar_electrolyzer_doe.yaml")
# Run the model
model.run()

Driver debug print for iter coord: rank0:DOEDriver_CSV|0
--------------------------------------------------------
Design Vars
{'electrolyzer.n_clusters': array([5.]), 'solar.capacity_kWdc': array([25000.])}

Nonlinear constraints
None

Linear constraints
None

Objectives
{'finance_subgroup_hydrogen.LCOH_optimistic': array([7.28340669])}

Driver debug print for iter coord: rank0:DOEDriver_CSV|1
--------------------------------------------------------
Design Vars
{'electrolyzer.n_clusters': array([5.]), 'solar.capacity_kWdc': array([50000.])}

Nonlinear constraints
None

Linear constraints
None

Objectives
{'finance_subgroup_hydrogen.LCOH_optimistic': array([4.71306895])}

Driver debug print for iter coord: rank0:DOEDriver_CSV|2
--------------------------------------------------------
Design Vars
{'electrolyzer.n_clusters': array([5.]),
 'solar.capacity_kWdc': array([100000.])}

Nonlinear constraints
None

Linear constraints
None

Objectives
{'finance_subgroup_hydrogen.LCOH_optimistic': 